In [2]:
#général
import pandas as pd
import numpy as np
import time

#traitement texte
import re

#visualisation
import matplotlib.pyplot as plt

In [4]:
# Récupération des données
df=pd.read_csv(r"C:\Users\John\Desktop\Formation\10-Analyser la charge des acteurs d'une pièce de théâtre\Le_malade_imaginairea.csv",encoding='latin1',sep=';',header=None)

In [16]:
df.iloc[0:10]

,0,1,2,3,4,5,6,7,8,9,...,227,228,229,230,231,232,233,234,235,236
0,ACTE,I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,SCENE,I.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,"ARGAN,",seul,dans,sa,chambre,"assis,",une,table,devant,"lui,",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,(Note,:,Julep,:,Terme,de,pharmacie.,Potion,adoucissante,ou,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


   if 'MONSIEUR' in str(df.iloc[r, :])==False:
                    data.append([0, 0, df.iloc[r, 0], 0])
                else:
                    data.append([0, 0, df.iloc[r, 0]+df.iloc[r, 1], 0])

In [30]:
#nettoyage de la base

#suppression de tous les signes de ponctuation sur la première colonne

#prérequis toutes les cellules doivent être au format texte
df[0].fillna('txt',inplace = True)
df[1].fillna('txt',inplace = True)

# Punctuation removal function
def remove_punct(text):
    """
        Remove the punctuation
    """
    return re.sub(r'[]!"$%&\'()*+,./:;=#@?[\\^_`{|}~-]+', "", text)

# Remove punctuations from the text
df[0] = df[0].apply(lambda x: remove_punct(x))
df[1] = df[1].apply(lambda x: remove_punct(x))
df.iloc[0:20]    

,0,1,2,3,4,5,6,7,8,9,...,227,228,229,230,231,232,233,234,235,236
0,ACTE,I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,txt,txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,txt,txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,txt,txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,txt,txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,SCENE,I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,txt,txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,txt,txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,ARGAN,seul,dans,sa,chambre,"assis,",une,table,devant,"lui,",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Note,,Julep,:,Terme,de,pharmacie.,Potion,adoucissante,ou,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
%%time
data = [['Acte', 'Scene', 'Personnage', 'Nb de mots']]
lnom = ['ARGAN', 'BELINE', 'TOINETTE', 'CLEANTE', 'BERALDE', 'DIAFOIRUS', 'ANGELIQUE', 'PURGON', 'LOUISON', 'MONSIEUR']
lstage=['ACTE','SCENE','PROLOGUE','PREMIER','SECOND','TROISIEME']
cw = 0
# On balaie le doc ligne par ligne
for r in df.index:
    # Vérifie si la cellule d'entrée de ligne contient le mot Acte ou Scène
    if df.iloc[r,0] in lstage :
        
        # Si c'est le cas crée une ligne dans le tableau avec son numéro
        data.append([f'{df.iloc[r, 0]} {df.iloc[r, 1]}', 0, 0, 0])
    
    # Sinon vérifie si la cellule d'entrée de ligne contient un nom
    elif df.iloc[r,0] in lnom :
        
        # Si c'est le cas crée une ligne avec le nom
        # Si le nom est MONSIEUR crée une ligne avec le nom MONSIEUR + xxx
        if df.iloc[r, 0] == 'MONSIEUR' : 
            data.append([0, 0, f'{df.iloc[r, 0]} {df.iloc[r, 1]}', 0])
        else :
            
            # Sinon crée une ligne avec le nom simple
            data.append([0, 0, f'{df.iloc[r, 0]}', 0])
                
    
    # Sinon crée une ligne avec le nombre de mots si la cellule d'entrée de ligne n'est pas 'txt'
    else : 
        cw = df.iloc[r, :].notna().sum()
        if df.iloc[r, 0] != 'txt' :
            data.append([0, 0, 0, cw])
            cw = 0
        cw = 0

data

CPU times: total: 875 ms
Wall time: 1.13 s


[['Acte', 'Scene', 'Personnage', 'Nb de mots'],
 ['ACTE I', 0, 0, 0],
 ['SCENE I', 0, 0, 0],
 [0, 0, 'ARGAN', 0],
 [0, 0, 0, 194],
 ['SCENE II', 0, 0, 0],
 [0, 0, 'TOINETTE', 0],
 [0, 0, 0, 3],
 [0, 0, 'ARGAN', 0],
 [0, 0, 0, 31],
 [0, 0, 'TOINETTE', 0],
 [0, 0, 0, 45],
 [0, 0, 'ARGAN', 0],
 [0, 0, 0, 4],
 [0, 0, 'TOINETTE', 0],
 [0, 0, 0, 2],
 [0, 0, 'ARGAN', 0],
 [0, 0, 0, 3],
 [0, 0, 'TOINETTE', 0],
 [0, 0, 0, 2],
 [0, 0, 'ARGAN', 0],
 [0, 0, 0, 5],
 [0, 0, 'TOINETTE', 0],
 [0, 0, 0, 2],
 [0, 0, 'ARGAN', 0],
 [0, 0, 0, 3],
 [0, 0, 'TOINETTE', 0],
 [0, 0, 0, 2],
 [0, 0, 'ARGAN', 0],
 [0, 0, 0, 33],
 [0, 0, 'TOINETTE', 0],
 [0, 0, 0, 13],
 [0, 0, 'ARGAN', 0],
 [0, 0, 0, 5],
 [0, 0, 'TOINETTE', 0],
 [0, 0, 0, 18],
 [0, 0, 'ARGAN', 0],
 [0, 0, 0, 2],
 [0, 0, 'TOINETTE', 0],
 [0, 0, 0, 5],
 [0, 0, 'ARGAN', 0],
 [0, 0, 0, 3],
 [0, 0, 'TOINETTE', 0],
 [0, 0, 0, 2],
 [0, 0, 'ARGAN', 0],
 [0, 0, 0, 3],
 [0, 0, 'TOINETTE', 0],
 [0, 0, 0, 2],
 [0, 0, 'ARGAN', 0],
 [0, 0, 0, 14],
 [0, 0, 'TOINE

In [62]:
dfr=pd.DataFrame(data=data[1:], columns=data[0])
dfr.to_csv('dfr.csv',index = False)
dfr

,Acte,Scene,Personnage,Nb de mots
0,ACTE I,0,0,0
1,SCENE I,0,0,0
2,0,0,ARGAN,0
3,0,0,0,194
4,SCENE II,0,0,0
...,...,...,...,...
2272,0,0,0,7
2273,0,0,0,16
2274,0,0,0,4
2275,0,0,0,19


In [63]:
# Mise en forme du tableau
# total du nombre de mots à chaque personnage

dfr = dfr[0:20]
for r in dfr.index:
    # si la cellule contien un nom
    if dfr.iloc[r, 2] != '0':
        # tant que les cellule du dessous ne contiennent pas de nom
        print('1')
        i = 1
        cw = 0
        while i > 0:
            try:

                if dfr.iloc[r + i, 2] == 0:
                    print('2')
                    # comptabiliser le nombre de mots
                    cw += dfr.iloc[r + i, 3]
                    # et l'ajouter à la ligne du nom
                    dfr.iloc[r, 3] = cw
                    i += 1
                else:
                    # sinon sortir de la boucle while
                    break
            except:
                break
dfr

1
2
1
1
2
2
1
2
1
1
2
1
1
2
1
1
2
1
1
2
1
1
2
1
1
2
1
1
2
1
1


,Acte,Scene,Personnage,Nb de mots
0,ACTE I,0,0,0
1,SCENE I,0,0,0
2,0,0,ARGAN,194
3,0,0,0,0
4,SCENE II,0,0,0
5,0,0,TOINETTE,3
6,0,0,0,3
7,0,0,ARGAN,31
8,0,0,0,31
9,0,0,TOINETTE,45


In [34]:
dfrr=dfr.copy()

In [35]:
for r in dfr.index:
    if 'SCENE' in str(dfr.iloc[r,0]):
        dfr.iloc[r,1]=dfr.iloc[r,0]
        dfr.iloc[r,0]=0

for r in dfr.index:
    if dfr.iloc[r,1]==0:
        try:
            dfr.iloc[r,1]=dfr.iloc[r-1,1]
        except:
            continue

for r in dfr.index:
    if dfr.iloc[r,0]==0:
        try:
            dfr.iloc[r,0]=dfr.iloc[r-1,0]
        except:
            continue

In [36]:
dfr

,Acte,Scene,Personnage,Nb de mots
0,ACTE I,0,0,0
1,ACTE I,SCENE I,0,0
2,ACTE I,SCENE I,ARGAN,0
3,ACTE I,SCENE I,0,194
4,ACTE I,SCENE II,0,0
...,...,...,...,...
2272,TROISIEME INTERMEDE,SCENE XIV,0,7
2273,TROISIEME INTERMEDE,SCENE XIV,0,16
2274,TROISIEME INTERMEDE,SCENE XIV,0,4
2275,TROISIEME INTERMEDE,SCENE XIV,0,19


In [13]:
dfr.iloc[:,2] = dfr.iloc[:,2].str.replace('\W', '', regex=True)

AttributeError: Can only use .str accessor with string values!

In [ ]:
dfinal=dfr[(dfr != 0).all(1)]

In [ ]:
dfp=dfinal.pivot_table(index='Personnage',columns='Acte',values='Nb de mots',aggfunc='sum')

In [ ]:
dfp

In [ ]:
dfinal=dfinal.loc[dfinal['Acte'].str.contains('ACTE'),:]

In [ ]:
plt.scatter(y=dfinal['Personnage'], x=dfinal['Acte'],s=5*dfinal['Nb de mots'])
#plt.scatter(y=dfp.index, x=dfp.iloc[:,1],s=dfp.iloc[:,1])
#plt.scatter(y=dfp.index, x=dfp.iloc[:,2],s=dfp.iloc[:,2])

In [ ]:
dfinal.groupby(['Acte','Personnage']).sum()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

test_date1, test_date2 = '20200101', '20210630'

months = pd.date_range('2021-01-01', periods=12, freq='M').strftime('%B')
K = 2000
df = pd.DataFrame(np.random.choice(pd.date_range(test_date1, test_date2), K), columns=['Upload Date'])
df['Year'] = df['Upload Date'].dt.year
# df['Month'] = pd.Categorical(df['Upload Date'].dt.strftime('%B'), categories=months)
df['YearMonth'] = df['Upload Date'].dt.strftime('%Y%m').astype(int)
df['DocID'] = np.random.randint(100000, 200000, df.shape[0]).astype('str')
df['DocID'] = 'DOC' + df['DocID']

sns.set_style("white")
sns.set_color_codes("pastel")
fig, ax = plt.subplots(figsize=(20, 8))
sns.countplot(x='YearMonth', data=df, ax=ax)
sns.despine()
yearmonth_labels = [int(l.get_text()) for l in ax.get_xticklabels()]
ax.set_xticklabels([months[ym % 100 - 1] for ym in yearmonth_labels])
ax.set_xlabel('')

# calculate the positions of the borders between the years
pos = []
years = []
prev = None
for i, ym in enumerate(yearmonth_labels):
    if ym // 100 != prev:
        pos.append(i)
        prev = ym // 100
        years.append(prev)
pos.append(len(yearmonth_labels))
pos = np.array(pos) - 0.5
# vertical lines to separate the years
ax.vlines(pos, 0, -0.12, color='black', lw=0.8, clip_on=False, transform=ax.get_xaxis_transform())
# years at the center of their range
for year, pos0, pos1 in zip(years, pos[:-1], pos[1:]):
    ax.text((pos0 + pos1) / 2, -0.07, year, ha='center', clip_on=False, transform=ax.get_xaxis_transform())

ax.set_xlim(pos[0], pos[-1])
ax.set_ylim(ymin=0)
plt.tight_layout()
plt.show()